<a href="https://colab.research.google.com/github/saleh1312/DeepFake/blob/main/write_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os 
import dlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import LeakyReLU
from skimage.transform import resize

In [ ]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat")

In [ ]:
d_s=tf.keras.models.load_model('/content/drive/MyDrive/encoder-da7ee7.h5', custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU(0.2)})

In [ ]:
def mask_p(img,mask):
  img= img.astype(np.float32)
  final = cv2.bitwise_and(img,img,mask=mask)
  final = cv2.resize(final,(128,128))
  mask = cv2.resize(mask,(128,128))
  final=(final.astype(np.float32))

  for x in range(mask.shape[1]):
    for y in range(mask.shape[0]):
      if mask[y,x] != 255:
        final[y,x]=(-1,-1,-1)

  return final


In [ ]:
def scar_to_da7(img):
  gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)
  faces = detector(gray)
  if len(faces) ==0:
    return faces
  p=None
  for face in faces:
    x1 = face.left() # left point
    y1 = face.top() # top point
    x2 = face.right() # right point
    y2 = face.bottom() # bottom point

    # Create landmark object
    landmarks = predictor(image=gray, box=face)

    # Loop through all the points
    points=[]
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        points.append((x,y))

    p=np.float32(points)
    break

  hull=cv2.convexHull(p,False)

  hullee=np.reshape(hull,(hull.shape[1],hull.shape[0],hull.shape[2]))

  mask=np.zeros((img.shape[0],img.shape[1]),np.uint8)
  cv2.fillPoly(mask,hullee.astype(np.int32),255)
  cnt, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  x,y,w,h = cv2.boundingRect(cnt[0])
  facee=img[y:y+h,x:x+w]
  maskee=mask[y:y+h,x:x+w]

  n=mask_p(facee,maskee)

  n=n/255.0
  
  for x1 in range(n.shape[1]):
    for y1 in range(n.shape[0]):
      if n[y1,x1][0] < 0 and n[y1,x1][1] < 0 and n[y1,x1][2] < 0:
        n[y1,x1]=(-1,-1,-1)

  t=np.reshape(n,(1,128,128,3)).astype(np.float32)
  o=d_s.predict(t)
  o=((((np.reshape(o,(128,128,3)).astype(np.float32)))))

  mask2=np.zeros((o.shape[0],o.shape[1],1),np.float32)

  o=o*255.0
  for x1 in range(o.shape[1]):
    for y1 in range(o.shape[0]):
      if o[y1,x1][0] > 0 and o[y1,x1][1] > 0 and o[y1,x1][2] > 0:
        mask2[y1,x1]=255.0


  mask_1=np.zeros((img.shape[0],img.shape[1],1),np.float32)
  mask_2=np.zeros((img.shape[0],img.shape[1],3),np.float32)

  o=resize(o,(h,w))
  mask2=resize(mask2,(h,w))


  mask_1[y:y+h,x:x+w]=mask2
  mask_2[y:y+h,x:x+w]=o


  for x1 in range(mask_1.shape[1]):
    for y1 in range(mask_1.shape[0]):
      if mask_1[y1,x1][0] == 255.0:
        img[y1,x1]=(mask_2[y1,x1]).astype(np.uint8)

  return img

    




In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/deepfake project/videos/1.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))

p=0
while True:
  if p==100:
    break
  ret,frame= cap.read()
  
  if ret:
    masks=scar_to_da7(frame) 
    
    if len(masks)==0:
      out.write(frame)
    else:
      out.write(masks)

  else:
    break
  p=p+1

cap.release()
out.release()

In [ ]:
cap.release()
out.release()